In [6]:
import os,sys,inspect
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )
testname="Тест неравномерности АЧХ"
display(Markdown('''### ░░░▄▀▀▀▄░ГУСЬ░

▄███▀░◐░░░▌░░░░░░░

░░░░▌░░░░░▐░░░░░░░

░░░░▐░░░░░▐░░░░░░░

░░░░▌░░░░░▐▄▄░░░░░

░░░░▌░░░░▄▀▒▒▀▀▀▀▄

░░░▐░░░░▐▒▒▒▒▒▒▒▒▀▀▄

░░░▐░░░░▐▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░▀▄░░░░▀▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░░░▀▄▄▄▄▄█▄▄▄▄▄▄▄▄▄▄▄▀▄

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░▄▄▌▌▄▌▌░░░░░ '''))
display(f'{testname}')

### ░░░▄▀▀▀▄░ГУСЬ░

▄███▀░◐░░░▌░░░░░░░

░░░░▌░░░░░▐░░░░░░░

░░░░▐░░░░░▐░░░░░░░

░░░░▌░░░░░▐▄▄░░░░░

░░░░▌░░░░▄▀▒▒▀▀▀▀▄

░░░▐░░░░▐▒▒▒▒▒▒▒▒▀▀▄

░░░▐░░░░▐▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░▀▄░░░░▀▄▒▒▒▒▒▒▒▒▒▒▀▄

░░░░░░▀▄▄▄▄▄█▄▄▄▄▄▄▄▄▄▄▄▀▄

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░░░▌▌░▌▌░░░░░

░░░░░░░░░▄▄▌▌▄▌▌░░░░░ 

'Тест неравномерности АЧХ'

In [ ]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)


In [10]:
button = box.select_device_continue_button

In [ ]:
get_ipython().run_line_magic('blockrun', 'button')

In [ ]:
button1 = box.select_config_continue_button

In [ ]:
get_ipython().run_line_magic('blockrun', 'button1')

In [ ]:
inst = box.selected_device

In [ ]:
inst.write("SYST:PRES")

In [ ]:
date=datetime.now()
date=date.strftime('%Y-%m-%d %H:%M:%S')
device = inst.query("*IDN?")
device = device.replace(' ','')
device = device.split(',')
device_info = {'Время запуска':date,
              'device':device[0],
              'model':device[1],
              'serial':device[2],
              'version':device[3]}
#inst2 = ng.instance()
#путь к файлу с сегментами
currentDir = get_ipython().run_line_magic('pwd', '')
#resultPath= currentDir +'\\frequency_plan.seg'
portCount=int(inst.query("SERV:PORT:COUN?"))
traces=["S","R","T"]
traces_count = portCount* len(traces)

In [ ]:
result_config =  box.config

In [ ]:
def get_ranges(ranges):
    result = {}
    arr=ranges.split(',')
    for i in range(len(arr)):
        result[i]={}        
        arr1 = arr[i].split('-')    
        result[i]['start'] = Convertor1(arr1[0])
        result[i]['stop'] = Convertor1(arr1[1])
    return result

In [ ]:
import datetime
import pyvisa
import json
import configparser
def Convertor1(value:str):    
    n,freq = value.split(' ')
    n=int(n)
    if freq == "kHz":
        return int(n*1e+3)
    elif freq == "MHz":
        return int(n*1e+6)
    elif freq == "GHz":
        return int(n*1e+9)
    else: return print("Incorrect value")


def get_ranges(ranges):
    result = {}
    arr=ranges.split(',')
    for i in range(len(arr)):
        result[i]={}        
        arr1 = arr[i].split('-')    
        result[i]['start'] = Convertor1(arr1[0])
        result[i]['stop'] = Convertor1(arr1[1])
    return result
def AntiConvertor(number):
    number1=int(number)
    # number1=str(number1)
    if (number1) < 1000000:
        return str(float(number1)/1000) + ' kHz'
    if (number1) < 1000000000:
        return str(float(number1)/1000000) + ' MHz'
    return str(float(number1)/1000000000) + ' GHz'
def multiplier_segments(INST):
    b = INST.query('SENS1:SEGM:DATA?').split(',')       
    # print(len(arr_end_mark))    
    for i in range(9, len(b), 6):
        b[i] = str(int(b[i]) * 2)   # домножение точек на 2
             # конечные
    upd_array = ','.join(b)
    INST.write(f'SENS1:SEGM:DATA {upd_array}')
def setup_chanel_parametrs(inst,att, ifbw):
    inst.write('SYST:PRES')
    inst.write('SERVice:RFCTL:POWer:STATe 1')
    inst.write(f'SERVice:RFCTL:POWer:ATT {att}')
    inst.write(f'SENS1:BWID {ifbw}')
    inst.write('SERV:RFCTL:POW:DAC 6554')
    inst.write('SERVice:SWEep:FREQuency:FACTory')
    multiplier_segments(inst)
def one_scan(INST):
    INST.write('TRIG:SOUR BUS')  # устанавливаем источник триггера на шину
    INST.write(f"INIT:CONT OFF")  # включаем повторный режим запуска триггера
    # переходим в состояние ожидания триггера
    INST.write('INIT')
    INST.write('TRIG:SING')  # выполняем россчерк
    INST.query('*OPC?')
def dict_gen(result):
    for key in ranges.keys():
        start = AntiConvertor(ranges[key]['start'])
        stop = AntiConvertor(ranges[key]['stop'])
        result['segment'][f'{start}-{stop}']={'absolute':{},'otnosit':{}}
        result['segment'][f'{start}-{stop}']['absolute']={f"S{j}{j}" : f"{0}" for j in range (1,trace_count+1)}
        T={f"T{j}{j}" : f"{0}" for j in range (1,trace_count+1)}
        R={f"R{j}{j}" : f"{0}" for j in range (1,trace_count+1)}
        result['segment'][f'{start}-{stop}']['otnosit']={**T,**R}
    return result
def setup_trace(inst, port, traces,ranges):
    for trace in traces:
        if trace is "S":
            inst.write(f'CALC1:PAR1:DEF {trace}{port}{port}')
            setup_markers(inst,1,ranges)
        elif trace is "T":
            inst.write(f'CALC1:PAR2:DEF {trace}{port}')
            inst.write(f'CALC1:PAR2:SPOR {port}')
            setup_markers(inst,2,ranges)
        else:
            inst.write(f'CALC1:PAR3:DEF {trace}{port}')
            inst.write(f'CALC1:PAR3:SPOR {port}')
            setup_markers(inst,3,ranges)
            
def setup_markers(inst,trace,ranges):
        inst.write(f'CALC1:PAR{trace}:SEL')
        marker_count=len(ranges)*2
        inst.write(f'CALC1:MARK{marker_count}:STAT ON')
        inst.write(f'CALC1:MST ON')
        inst.write(f'CALC1:MST:DOM ON')
        for key in ranges.keys():
            # ((key+1) * 2 ) - 1 start
            # # ((key+1) * 2 ) stop
            inst.write(f'CALC1:MARK{((key+1) * 2 ) - 1}:X {ranges[key]["start"]}')
            inst.write(f'CALC1:MARK{((key+1) * 2 )}:X {ranges[key]["stop"]}')
def get_data(inst,port,result,start,stop):

    for i in range(1,4):
        inst.write(f'CALC1:PAR{i}:SEL')
   
        inst.write(f'CALC:MST:DOM:START {((key+1) * 2 ) - 1}')
        inst.write(f'CALC:MST:DOM:STOP {((key+1) * 2 )}')
        mst_data=inst.query(f'CALC1:MST:DATA?')
        value = mst_data.split(',')
        value = format(float(value[2]),'.8f')
        if i is 1:
                result['segment'][f'{start}-{stop}']['absolute'][f'S{port}{port}']=value
        elif i is 2:
                result['segment'][f'{start}-{stop}']['otnosit'][f'T{port}{port}']=value
        else:
                result['segment'][f'{start}-{stop}']['otnosit'][f'R{port}{port}']=value
    return result



In [ ]:
start_time = datetime.datetime.now()
config = box.config
ranges = get_ranges(config['range']['ranges'])
IFBW = Convertor1(config['options']['IFBW'])
attenuator = config['options']['attenuator']
traces = ['S','T','R']
setup_chanel_parametrs(inst,attenuator,IFBW)
trace_count=int(inst.query(f'SERV:PORT:COUNT?'))
inst.write(f'CALC1:PAR:COUN 3')
result=dict_gen(result={"segment":{}})

In [ ]:
for port in range(1,trace_count+1):   
    setup_trace(inst,port,traces,ranges)
    one_scan(inst)
    for key in ranges.keys():
        start = AntiConvertor(ranges[key]['start'])
        stop = AntiConvertor(ranges[key]['stop'])
        result=get_data(inst,port,result,start,stop)
for key in ranges.keys():
    start = AntiConvertor(ranges[key]['start'])
    stop = AntiConvertor(ranges[key]['stop'])
    result['segment'][f'{start}-{stop}']['absolute']['max_abs']=max(result['segment'][f'{start}-{stop}']['absolute'].values())
    result['segment'][f'{start}-{stop}']['otnosit']['max_diff']=max(result['segment'][f'{start}-{stop}']['otnosit'].values())

In [ ]:
def json_html(result):
    config = box.config
#     config = configparser.ConfigParser()
#     config.read('config.cfg')
    dop = config['options']['dop']

    rm = pyvisa.ResourceManager('@py')
    try:
        CMT = rm.open_resource('TCPIP0::localhost::5025::SOCKET')
    except:
        print("Failure to connect to VNA!")
        print("Check network settings")
    CMT.read_termination = '\n'
    CMT.timeout = 10000
    data = result
    start_time = datetime.datetime.now()
    perf_data=CMT.query(f'*IDN?').split(',')
    trace_count=int(CMT.query(f'SERV:PORT:COUNT?'))
    sko_result = open('index3.html', 'w')
    htmlstring = f'''
    <table style = "width:100%; margin-left:auto;margin-right:auto;">
    <tr>
        <th>
            Время запуска
        </th>
        <th>
            Девайс
        </th>
        <th>
            Модель
        </th>
        <th>
            Серийник
        </th>
        <th>
            Версия
        </th>
        </tr>
            
        <tr>
            <td>
                {start_time}
            </td>
            <td>
                {perf_data[0]}
            </td>
            <td>
                {perf_data[1]}
            </td>
            <td>
                {perf_data[2]}
            </td>
            <td>
                {perf_data[3]}
            </td>
        </tr>
    </table>
    '''
    htmlstring+='''
    <style>
        table {
            font-family: "Lucida Sans Unicode", "Lucida Grande", Sans-Serif;
            font-size: 14px;
            border-collapse: collapse;
            text-align: center;
        }
        th{border: 1px solid gold;}
        td{border: 1px solid gold;}
    </style>'''
    param=['S','T','R']
    value=''
    value1=''
    value2=''
    value3=''
    for key in data['segment'].keys():
        value+=(f'<th colspan="3">{key}</th>')
        value1+=(f'<td>Неровность</td><td>Макс. значение</td><td>Допустимое значение</td>')
    for name in param:
        for port in range(1,trace_count+1):
            for key in data['segment'].keys():
                if name is 'S':
                    if data["segment"][key]["absolute"][f"{name}{port}{port}"] == data["segment"][key]["absolute"]["max_abs"]:
                        value3+=f'<td>{data["segment"][key]["absolute"][f"{name}{port}{port}"]}</td><td bgcolor="pink" >{data["segment"][key]["absolute"]["max_abs"]}</td><td bgcolor="pink">{dop}</td>'
                    else:
                        value3+=f'<td>{data["segment"][key]["absolute"][f"{name}{port}{port}"]}</td><td>-</td><td>{dop}</td>'
                else:
                    if data["segment"][key]["otnosit"][f"{name}{port}{port}"] == data["segment"][key]["otnosit"]["max_diff"]:
                        value3+=f'<td>{data["segment"][key]["otnosit"][f"{name}{port}{port}"]}</td><td bgcolor="pink">{data["segment"][key]["otnosit"]["max_diff"]}</td><td bgcolor="pink">{dop}</td>'
                    else:
                         value3+=f'<td>{data["segment"][key]["otnosit"][f"{name}{port}{port}"]}</td><td>-</td><td>{dop}</td>'
            value2+=f'<tr><td>{name}{port}{port}</td>{value3}</tr>'
            value3 = ""
    htmlstring1=f'''<table style="width:100%; margin-left:auto;margin-right:auto;">
        <tr>
        <th>
            Диапазон
        </th>
            {value}
        </tr>
        <tr>
            <td>Трассы</td>
            {value1}
        </tr>
        {value2}
    </table>'''
#     print(htmlstring+htmlstring1)
#     sko_result = open('index3.html', 'w')
#     sko_result.write(htmlstring+htmlstring1)
#     sko_result.close()
    return(htmlstring+htmlstring1)

NameError: name 'json_html' is not defined

In [ ]:
from IPython.core.display import display, HTML
html_table=json_html(result)


display(HTML(html_table))


In [ ]:
file_data = html_table
my_button = SaveFilesButton(file_data)
my_button # This will display the button in the context of Jupyter Notebook